In [1]:
# vmi vegetation rasters into SpaFHy vegetation rasters
# meaning that needle biomasses need to be converted to LAI
# unit conversion for e.g. canopy height
# saving them in .asc

In [2]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import rasterio

ch_file = r'/Users/jpnousu/WBT_data/rasters/vmi_16/keskipituus_vmi1x_1721.asc'
bmleaf_pine_file = r'/Users/jpnousu/WBT_data/rasters/vmi_16/bm_manty_neulaset_vmi1x_1721.asc'
bmleaf_spruce_file = r'/Users/jpnousu/WBT_data/rasters/vmi_16/bm_kuusi_neulaset_vmi1x_1721.asc'
bmleaf_decid_file = r'/Users/jpnousu/WBT_data/rasters/vmi_16/bm_lehtip_neulaset_vmi1x_1721.asc'
cf_file = r'/Users/jpnousu/WBT_data/rasters/vmi_16/latvuspeitto_vmi1x_1721.asc'

# --- Canopy height ---
with rasterio.open(ch_file) as src:
    ch = src.read(1)
    ch_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    ch = np.where(ch == old_nodata, np.nan, ch)
ch_meta.update({"nodata": -9999, "dtype": "float32"})

# --- Biomass pine ---
with rasterio.open(bmleaf_pine_file) as src:
    bmleaf_pine = src.read(1)
    bmleaf_pine_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    bmleaf_pine = np.where(bmleaf_pine == old_nodata, np.nan, bmleaf_pine)
bmleaf_pine_meta.update({"nodata": -9999, "dtype": "float32"})

# --- Biomass spruce ---
with rasterio.open(bmleaf_spruce_file) as src:
    bmleaf_spruce = src.read(1)
    bmleaf_spruce_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    bmleaf_spruce = np.where(bmleaf_spruce == old_nodata, np.nan, bmleaf_spruce)
bmleaf_spruce_meta.update({"nodata": -9999, "dtype": "float32"})

# --- Biomass deciduous ---
with rasterio.open(bmleaf_decid_file) as src:
    bmleaf_decid = src.read(1)
    bmleaf_decid_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    bmleaf_decid = np.where(bmleaf_decid == old_nodata, np.nan, bmleaf_decid)
bmleaf_decid_meta.update({"nodata": -9999, "dtype": "float32"})

# --- Canopy fraction ---
with rasterio.open(cf_file) as src:
    cf = src.read(1)
    cf_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    cf = np.where(cf == old_nodata, np.nan, cf)
cf_meta.update({"nodata": -9999, "dtype": "float32"})

In [4]:
# Specific leaf area (m2/kg)
SLA = {'pine': 6.8, 'spruce': 4.7, 'decid': 14.0}  # Härkönen et al. 2015 BER 20, 181-195

# Convert biomass to kg/m2
bmleaf_pine = (1e-3 * bmleaf_pine)
bmleaf_spruce = (1e-3 * bmleaf_spruce)
bmleaf_decid = (1e-3 * bmleaf_decid)

# Convert canopy height dm -> m
ch = (0.1 * ch).astype(np.float32)

# Convert crown fraction % -> [0,1]
cf = (1e-2 * cf).astype(np.float32)

# Compute LAI
LAI_pine = bmleaf_pine * SLA['pine']
LAI_spruce = bmleaf_spruce * SLA['spruce']
LAI_decid = bmleaf_decid * SLA['decid']

small_val = 0.001
ch = np.nan_to_num(ch, nan=small_val)
cf = np.nan_to_num(cf, nan=small_val)
LAI_pine = np.nan_to_num(LAI_pine, nan=small_val)
LAI_spruce = np.nan_to_num(LAI_spruce, nan=small_val)
LAI_decid = np.nan_to_num(LAI_decid, nan=small_val)

LAI_pine[LAI_pine < small_val] = small_val
LAI_spruce[LAI_spruce < small_val] = small_val
LAI_decid[LAI_decid < small_val] = small_val

In [5]:
# --- Canopy height ---
out_ch_file = "/Users/jpnousu/WBT_data/rasters/vmi_16/canopy_height.asc"
with rasterio.open(out_ch_file, "w", **ch_meta) as dst:
    dst.write(ch, 1)

# --- Canopy fraction ---
out_cf_file = "/Users/jpnousu/WBT_data/rasters/vmi_16/canopy_fraction.asc"
with rasterio.open(out_cf_file, "w", **cf_meta) as dst:
    dst.write(cf, 1)

# --- LAI Pine ---
out_LAI_pine_file = "/Users/jpnousu/WBT_data/rasters/vmi_16/LAI_pine.asc"
with rasterio.open(out_LAI_pine_file, "w", **ch_meta) as dst:  # use ch_meta for spatial info
    dst.write(LAI_pine, 1)

# --- LAI Spruce ---
out_LAI_spruce_file = "/Users/jpnousu/WBT_data/rasters/vmi_16/LAI_spruce.asc"
with rasterio.open(out_LAI_spruce_file, "w", **ch_meta) as dst:
    dst.write(LAI_spruce, 1)

# --- LAI Deciduous ---
out_LAI_decid_file = "/Users/jpnousu/WBT_data/rasters/vmi_16/LAI_decid.asc"
with rasterio.open(out_LAI_decid_file, "w", **ch_meta) as dst:
    dst.write(LAI_decid, 1)